In [10]:
!pip install vosk

Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install gtts pygame

Defaulting to user installation because normal site-packages is not writeable


<h1>TRANSCRIPCIÓN DE VOZ A TEXTO / PARA LLENAR UNA PLANTILLA EN PDF.</h1>

**<h2>Universidad Nacional de Loja</h2>**

----

 **<h3>Materia:** Human Perception in Computer Vision</h3>
  **<h3>Integrantes:**</h3>
+ Luis Bermeo
+ Anthony Luzuriaga
+ Gerar Quizhpe
+ Alex Viñamagua

In [60]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'FICHA DE DESCRIPCIÓN DE AFLORAMIENTOS', 0, 1, 'C')

    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')

    def chapter_body(self, body):
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln()

    def table(self, data):
        self.set_font('Arial', 'B', 12)
        # Header, el total es de 190 el largo del documento. osea 19cm.
        self.cell(40, 10, 'Responsable:', 1)
        self.cell(150, 10, data['responsable'], 1)
        self.ln()
        self.cell(40, 10, 'Afloramiento N°:', 1)
        self.cell(150, 10, data['afloramiento'], 1)
        self.ln()
        self.cell(40, 10, 'Carta Topográfica:', 1)
        self.cell(150, 10, data['carta_topografica'], 1)
        self.ln()
        ######################################################################
        self.cell(190, 10, 'UBICACIÓN', 1, 1, 'C')
        self.cell(30, 10, 'Provincia:', 1)
        self.cell(160, 10, data['provincia'], 1)
        self.ln()
        self.cell(30, 10, 'Cantón:', 1)
        self.cell(160, 10, data['canton'], 1)
        self.ln()
        self.cell(30, 10, 'Parroquia:', 1)
        self.cell(160, 10, data['parroquia'], 1)
        self.ln()
        self.cell(30, 10, 'Sector:', 1)
        self.cell(160, 10, data['sector'], 1)
        self.ln()
        self.cell(30, 10, 'DATUM:', 1)
        self.cell(160, 10, "WGS 1984", 1)
        self.ln()
        #######################################################################
        self.cell(190, 10, 'Localización', 1, 1, 'C')
        self.cell(30, 10, 'X:', 1)
        self.cell(160, 10, data['localizacion_x'], 1)
        self.ln()
        self.cell(30, 10, 'Y:', 1)
        self.cell(160, 10, data['localizacion_y'], 1)
        self.ln()
        self.cell(30, 10, 'Z:', 1)
        self.cell(160, 10, str(data['localizacion_z']) + ' m', 1)
        self.ln()
        ########################################################################
        self.cell(190, 10, 'Tipo de afloramiento', 1, 1, 'C')
        self.cell(45, 10, 'Alforamiento Natural:', 1)
        self.cell(45, 10, data['tipo_afloramiento_natural'], 1)
        self.cell(55, 10, 'Alforamiento Antropogenico:', 1)
        self.cell(45, 10, data['tipo_afloramiento_antropogenico'], 1)
        self.ln()
        ########################################################################
        self.cell(190, 10, 'Dimensiones', 1, 1, 'C')
        self.cell(20, 10, 'Largo:', 1)
        self.cell(75, 10, str(data['largo']) + ' m', 1)
        self.cell(20, 10, 'Ancho:', 1)
        self.cell(75, 10, str(data['ancho']) + ' m', 1)
        self.ln()


In [20]:
# Función para crear y guardar el PDF
def create_pdf(data, output_path):
    pdf = PDF()
    pdf.add_page()
    pdf.table(data)
    pdf.output(output_path)

In [21]:
import os
import wave
import json
import vosk
import pyaudio
import time
import pygame
from gtts import gTTS

In [22]:
# Inicializar Pygame
pygame.mixer.init()

In [23]:
def play_sound(file):
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

In [24]:
def text_to_audio(text, filename):
    tts = gTTS(text=text, lang='es')
    tts.save(filename)

In [25]:
import tempfile

def play_text(text):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
        temp_filename = temp_audio.name
        text_to_audio(text, temp_filename)
    
    # Asegurarse de que el archivo se ha guardado antes de cargarlo
    if os.path.exists(temp_filename):
        pygame.mixer.music.load(temp_filename)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)
        time.sleep(0.5)  # Pequeño retraso para asegurarse de que Pygame ha terminado de usar el archivo
        try:
            os.remove(temp_filename)
        except PermissionError:
            print(f"Error: No se pudo eliminar el archivo {temp_filename}")
    else:
        print(f"Error: No se pudo encontrar el archivo {temp_filename}")

In [48]:
def convertir_a_numero(texto):
    # Diccionarios para mapeo de palabras a números
    unidades = {
        'uno': 1, 'un': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5,
        'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
        'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
        'dieciséis': 16, 'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19, 'veinte': 20,
        'veintiuno': 21, 'veintidós': 22, 'veintitrés': 23, 'veinticuatro': 24, 'veinticinco': 25,
        'veintiséis': 26, 'veintisiete': 27, 'veintiocho': 28, 'veintinueve': 29
    }
    decenas = {
        'treinta': 30, 'cuarenta': 40, 'cincuenta': 50, 'sesenta': 60,
        'setenta': 70, 'ochenta': 80, 'noventa': 90
    }
    centenas = {
        'ciento': 100, 'doscientos': 200, 'trescientos': 300, 'cuatrocientos': 400, 'quinientos': 500,
        'seiscientos': 600, 'setecientos': 700, 'ochocientos': 800, 'novecientos': 900
    }
    especiales = {
        'cien': 100, 'mil': 1000, 'millón': 1000000, 'millones': 1000000, 'cero': 0
    }

    palabras = texto.split()
    total = 0
    temporal = 0
    resultado = ""
    decimal_part = False

    for palabra in palabras:
        if palabra in unidades:
            if decimal_part:
                resultado += str(unidades[palabra])
            else:
                temporal += unidades[palabra]
        elif palabra in decenas:
            temporal += decenas[palabra]
        elif palabra in centenas:
            temporal += centenas[palabra]
        elif palabra in especiales:
            if palabra == 'mil':
                if temporal == 0:
                    temporal = 1
                total += temporal * 1000
                temporal = 0
            elif palabra == 'millón' or palabra == 'millones':
                if temporal == 0:
                    temporal = 1
                total += temporal * 1000000
                temporal = 0
            else:
                temporal += especiales[palabra]
        elif palabra == 'punto' or palabra == 'coma':  # Manejar separadores decimales
            if not decimal_part:
                resultado += str(total + temporal) + "."
                decimal_part = True
                temporal = 0
        elif palabra == 'y':
            continue
        else:
            return "0"

    if not decimal_part:
        resultado += str(total + temporal)
    return resultado

In [27]:
# Función para capturar voz

def capture_audio():
    p = pyaudio.PyAudio()
    # Abre un flujo de audio con los siguientes parámetros:
    # - formato: 16 bits por muestra
    # - canales: 1 (mono)
    # - tasa de muestreo: 16000 Hz
    # - tamaño del buffer: 8192
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8192)
    stream.start_stream()
    
    #https://alphacephei.com/vosk/models
    model = vosk.Model("model/vosk-model-small-es-0.42")
     # Crea un reconocedor de Kaldi con el modelo cargado y la tasa de muestreo
    rec = vosk.KaldiRecognizer(model, 16000)
    
    print("Di tu comando:")
    #play_text("Di tu comando")
    
    while True:
        # Lee datos del flujo de audio
        data = stream.read(4096)
        # Procesa los datos de audio y verifica si se ha reconocido una forma de onda completa
        if rec.AcceptWaveform(data):
            # Si se reconoce una forma de onda completa, obtiene el resultado en formato JSON
            result = rec.Result()
            text = json.loads(result)["text"]
            break
    # Detiene y cierra el flujo de audio
    stream.stop_stream()
    stream.close()
    p.terminate()
 
    return text

In [47]:
# Función para asignar valores a variables
def assign_variable(variable_name):
    
    #print(f"Por favor, di el valor para {variable_name}:")
    prompt = f"Por favor, di el valor para {variable_name}:"
    #play_sound(f"audio/{variable_name}.mp3")
    play_text(prompt)
    
    value = capture_audio()
    
    #print(f"Capturado: {value}")  # Verifica que se capturó el valor correcto
    captured_message = f"Capturado: {value}"
    #play_sound("audio/capturado.mp3")
    play_text(captured_message)
    
    if variable_name in ['responsable', 'localizacion_x', 'localizacion_y', 'localizacion_z', 'largo', 'ancho']:
        value = convertir_a_numero(value)
    
        # Manejar el caso especial para el tipo de afloramiento
    if variable_name in ['tipo_afloramiento_natural', 'tipo_afloramiento_antropogenico']:
        if value.lower() == 'si':
            value = 'X'
        else:
            value = ''
            
    print(value)
    data[variable_name] = value

In [40]:
# Diccionario para guardar variables
data = {
    'responsable': '',
    'afloramiento': '',
    'carta_topografica': '',
    'provincia': '',
    'canton': '',
    'parroquia': '',
    'sector': '',
    'localizacion_x': '',
    'localizacion_y': '',
    'localizacion_z': '',
    'tipo_afloramiento_natural': '',
    'tipo_afloramiento_antropogenico': '',
    'largo': '',
    'ancho': '',
}

In [41]:
def main():
    for key in data.keys():
        assign_variable(key)

In [57]:
if __name__ == "__main__":
    
    main()

Error: No se pudo eliminar el archivo C:\Users\LENOVO\AppData\Local\Temp\tmpjbudw2jr.mp3
Di tu comando:
Error: No se pudo eliminar el archivo C:\Users\LENOVO\AppData\Local\Temp\tmpsu908e26.mp3
27.3


KeyboardInterrupt: 

In [58]:
output_path = 'plantilla_ejemplo_datos.pdf'
create_pdf(data, output_path)